In [37]:
# | default_exp routes.group

In [38]:
# | exporti
import httpx
from enum import Enum
from typing import Union

import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda
import domolibrary.client.DomoError as de

# Search and Get Routes


In [39]:
# | export
class SearchGroups_Error(de.DomoError):
    def __init__(
        self, status, message, domo_instance, function_name="search_groups_by_name"
    ):
        super().__init__(
            function_name=function_name,
            status=status,
            message=message,
            domo_instance=domo_instance,
        )


async def search_groups_by_name(
    auth: dmda.DomoAuth,
    search_name: str,
    is_exact_match: bool = True,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
) -> rgd.ResponseGetData:
    """uses /content/v2/groups/grouplist api -- includes user details"""

    url = f"https://{auth.domo_instance}.domo.com/api/content/v2/groups/grouplist?ascending=true&search={search_name}&sort=name "

    res = await gd.get_data(
        auth=auth, url=url, method="GET", debug_api=debug_api, session=session
    )
    if not is_exact_match:
        return res

    match_group = next(
        (group for group in res.response if group.get("name") == search_name), None
    )
    # print(match_group)

    if not match_group:
        raise SearchGroups_Error(
            status=res.status,
            message=f"There is no exact match for {search_name}",
            domo_instance=auth.domo_instance,
        )
    res.response = match_group

    return res

#### Sample implementation of search_groups_by_name


In [40]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)
search_name = "Test Group 2"
try:
    res = await search_groups_by_name(
        auth=token_auth, search_name=search_name, debug_api=False
    )
    print(res)
except SearchGroups_Error as e:
    print(e)
except Exception as e:
    print(e)

ResponseGetData(status=200, response={'name': 'Test Group 2', 'groupId': 1259653287, 'owners': [{'type': 'GROUP', 'id': '822382906', 'displayName': 'Grant: Manage all groups'}, {'type': 'USER', 'id': '1893952720', 'displayName': 'Jae Wilson1'}, {'type': 'GROUP', 'id': '1259653287', 'displayName': 'Test Group 2'}, {'type': 'GROUP', 'id': '2146122228', 'displayName': 'test'}, {'type': 'USER', 'id': '1681443709', 'displayName': 'test 3'}], 'groupType': 'open', 'groupMembers': [{'type': 'USER', 'id': '2072616249'}, {'type': 'USER', 'id': '663516735'}], 'memberCount': 2, 'created': '2023-02-15 17:10:37.0', 'description': ''}, is_success=True)


In [41]:
# | export


async def get_all_groups(
    auth: dmda.DomoAuth, debug_api: bool = False, session: httpx.AsyncClient = None
) -> rgd.ResponseGetData:
    """uses /content/v2/groups/grouplist api -- includes user details"""

    url = f"https://{auth.domo_instance}.domo.com/api/content/v2/groups/grouplist"

    res = await gd.get_data(
        url=url, method="GET", auth=auth, debug_api=debug_api, session=session
    )

    return res

#### Sample implementation of get_all_groups


In [42]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

res = await get_all_groups(auth=token_auth)
res_df = pd.DataFrame(res.response)
res_df[0:1]

,name,groupId,owners,groupType,groupMembers,memberCount,created,description
0,Admin Test,1814479647,"[{'type': 'USER', 'id': '55874022', 'displayNa...",closed,"[{'type': 'USER', 'id': '55874022'}, {'type': ...",2,2023-01-29 23:37:13.0,


In [43]:
# | export
async def get_group_by_id(
    auth: dmda.DomoAuth,
    group_id: str,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
) -> rgd.ResponseGetData:

    """uses /content/v2/groups/ api -- does not return details"""

    url = f"https://{auth.domo_instance}.domo.com/api/content/v2/groups/{group_id}"

    res = await gd.get_data(
        auth=auth, url=url, method="GET", debug_api=debug_api, session=session
    )

    if res.status == 404 and res.response == "Not Found":
        raise SearchGroups_Error(
            status=res.status,
            message=f"group {group_id} not found",
            domo_instance=auth.domo_instance,
            function_name="get_group_by_id",
        )

    return res

In [44]:
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

await get_group_by_id(auth=token_auth, group_id=1259653287)

ResponseGetData(status=200, response={'id': 1259653287, 'name': 'Test Group 2', 'type': 'open', 'userIds': [2072616249, 663516735], 'creatorId': 1893952720, 'memberCount': 2, 'guid': 'aabb6587-ad53-11ed-82c3-0a09ba383c95', 'description': '', 'hidden': False, 'default': False, 'active': True}, is_success=True)

# CRUD Routes


In [45]:
# | export
class GroupType_Enum(Enum):
    OPEN = "open"
    ADHOC = "adHoc"
    CLOSED = "closed"
    DIRECTORY = "directory"
    DYNAMIC = "dynamic"
    SYSYTEM = "system"


def generate_body_create_group(
    group_name: str, group_type: str = "open", description: str = ""
) -> dict:
    """Generates the body to create group for content_v2_group API"""
    body = {"name": group_name, "type": group_type, "description": description}

    return body

#### sample_implementation of generate_body_create_group


In [46]:
generate_body_create_group(
    group_name="test_group_name",
    group_type=GroupType_Enum.ADHOC.value,
    description="from jupyter",
)

{'name': 'test_group_name', 'type': 'adHoc', 'description': 'from jupyter'}

In [47]:
# | export
class CreateGroup_Error(de.DomoError):
    def __init__(self, status, message, domo_instance, function_name="create_group"):
        super().__init__(
            function_name=function_name,
            status=status,
            message=message,
            domo_instance=domo_instance,
        )


async def create_group(
    auth: dmda.DomoAuth,
    group_name: str,
    group_type: str = "open",
    description: str = "",
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
) -> rgd.ResponseGetData:
    # body : {"name": "GROUP_NAME", "type": "open", "description": ""}

    body = generate_body_create_group(
        group_name=group_name, group_type=group_type, description=description
    )

    url = f"https://{auth.domo_instance}.domo.com/api/content/v2/groups/"

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="POST",
        body=body,
        debug_api=debug_api,
        session=session,
    )

    if not res.is_success:
        group_exists = await search_groups_by_name(
            auth=auth, search_name=group_name, is_exact_match=True
        )
        if group_exists.is_success:
            raise CreateGroup_Error(
                status=res.status,
                message=f"{group_name} already exists. Choose a different group_name",
                domo_instance=auth.domo_instance,
                function_name="create_group",
            )

    if not res.is_success:
        raise CreateGroup_Error(
            status=res.status,
            message=res.response,
            domo_instance=auth.domo_instance,
            function_name="create_group",
        )

    return res

#### Sample implementation of create_group


In [48]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)
try:
    res = await create_group(
        auth=token_auth, group_name="Test Group ABC", debug_api=False
    )
    res.response

except CreateGroup_Error as e:
    print(e)

create_group: Status 400 - Test Group ABC already exists. Choose a different group_name at domo-community


In [49]:
# | export
async def update_group(
    auth: dmda.DomoAuth,
    group_id: int,
    group_name: str = None,
    group_type: str = None,
    description: str = None,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
) -> rgd.ResponseGetData:

    body = [
        {
            "groupId": int(group_id),
            "name": group_name,
            "type": group_type,
            "description": description,
        }
    ]

    url = f"https://{auth.domo_instance}.domo.com/api/content/v2/groups/"

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="PUT",
        body=body,
        debug_api=debug_api,
        session=session,
    )

    return res

#### sample implementation of update_group


In [50]:
import os
import datetime as dt

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

group_id = 250707376

await update_group(
    auth=token_auth,
    group_id=group_id,
    group_name="Test Group ABC",
    description=f"updated via API on {dt.date.today()}",
    debug_api=False,
)

ResponseGetData(status=200, response='', is_success=True)

# Group Membership
## GET  Group Ownership

In [51]:
# | export
async def get_group_owners(
    auth: dmda.DomoAuth,
    group_id: str,
    return_raw: bool = False,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
) -> rgd.ResponseGetData:

    # url = f"https://{auth.domo_instance}.domo.com/api/content/v2/groups/access"
    # url = f"https://{auth.domo_instance}.domo.com/api/content/v2/groups/users?group={group_id}"
    url = f'https://{auth.domo_instance}.domo.com/api/content/v2/groups/permissions?checkOwnership=true&includeUsers=false'

    if debug_api:
        print(url, body)

    res = await gd.get_data(
        auth=auth,
        url=url,
        body = [str(group_id)],
        method="POST",
        debug_api=debug_api,
        session=session,
    )

    if return_raw:
        return res

    res.response = res.response[0].get('permissions').get('owners')
    return res



#### sample implementation of get_ownership

In [52]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

group_id = 1781661643

res = await get_group_owners(auth=token_auth,
                           group_id=group_id,
                           )

pd.DataFrame(res.response)

,type,id,displayName
0,USER,1893952720,Jae Wilson1
1,GROUP,1781661643,TestMembership
2,GROUP,822382906,Grant: Manage all groups


In [53]:
# | export
async def get_group_membership(
    auth: dmda.DomoAuth,
    group_id: str,
    return_raw: bool = False,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
) -> rgd.ResponseGetData:

    # url = f"https://{auth.domo_instance}.domo.com/api/content/v2/groups/access"
    url = f"https://{auth.domo_instance}.domo.com/api/content/v2/groups/users?group={group_id}"

    if debug_api:
        print(url, body)

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="GET",
        debug_api=debug_api,
        session=session,
    )

    if return_raw:
        return res

    res.response = res.response.get('groupUserList')
    return res

#### sample implementation of get_membership

In [54]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

group_id = 1781661643

res = await get_group_membership(auth=token_auth,
                           group_id=group_id,
                           )

pd.DataFrame(res.response)

,userId,role,roleId,lastSignIn,location
0,696468809,Dojo_Default_Priviliged,2097317660,2021-04-19 16:36:26.944,NaN
1,1345737456,Privileged,2,2023-04-06 06:21:16.0,
2,1141078945,Dojo_Default_Priviliged,2097317660,2021-04-28 03:58:56.0,NaN


## CRUD Group Membership

In [55]:
def generate_body_update_group_membership_entity(id: Union[str, int],
                                                 type: str  # USER or GROUP
                                                 ):
    if type == 'USER':
        return {"type": "USER", "id": str(id)}
    elif type == 'GROUP':
        return {"type": "GROUP", "id": int(id)}


In [56]:
# | export
def generate_body_update_group_membership(group_id: str,
                                          add_member_arr: list[str] = None,
                                          remove_member_arr: list[str] = None,

                                          add_owner_arr: list[str] = None,
                                          remove_owner_arr: list[str] = None) -> list[dict]:
    """
    each member or owner obj should be an object of shape {"type", "id"}
    """

    body = {"groupId": int(group_id)}

    if add_owner_arr:
        body.update({"addOwners": [generate_body_update_group_membership_entity(
            id=obj.get('id'), type=obj.get('type')) for obj in add_owner_arr]})

    if remove_owner_arr:
        body.update({"removeOwners": [generate_body_update_group_membership_entity(
            id=obj.get('id'), type=obj.get('type')) for obj in remove_owner_arr]})

    if remove_member_arr:
        body.update({"removeMembers": [
                    generate_body_update_group_membership_entity(id=obj.get('id'), type=obj.get('type')) for obj in remove_member_arr]})
    if add_member_arr:
        body.update(
            {"addMembers": [generate_body_update_group_membership_entity(id=obj.get('id'), type=obj.get('type')) for obj in add_member_arr]})

    return [body]


In [57]:
# | export
async def update_group_membership(
    auth: dmda.DomoAuth,
    group_id: str,
    add_member_arr: list[dict] = None,
    remove_member_arr: list[dict] = None,
    add_owner_arr: list[dict] = None,
    remove_owner_arr: list[dict] = None,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
) -> rgd.ResponseGetData:

    """
    each member or owner obj should be an object of shape {"type", "id"}
    """

    body = generate_body_update_group_membership(group_id = group_id,
                                                 add_member_arr = add_member_arr,
                                                 remove_member_arr= remove_member_arr,
                                                 add_owner_arr = add_owner_arr,
                                                 remove_owner_arr = remove_owner_arr)

    # body = [{
    #     "groupId":"GROUP_ID",
    #     "removeMembers": [{"type":"USER","id":"USER_ID"}],
    #     "addMembers"   : [{"type":"USER","id":"USER_ID"}]
    # }]
    url = f"https://{auth.domo_instance}.domo.com/api/content/v2/groups/access"

    if debug_api:
        print(url, body)

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="PUT",
        body=body,
        debug_api=debug_api,
        session=session,
    )

    return res


#### Sample implementation of generate_body_update_group_membership and update_group_membership


In [58]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

group_id = 250707376

add_user_obj = {'id':  os.environ["DOMO_DOJO_USER_ID"],
                'type': 'USER'}

await update_group_membership(auth=token_auth, 
                        group_id=group_id,
                                    add_member_arr=[add_user_obj], 
                                    add_owner_arr=[add_user_obj])


ResponseGetData(status=200, response='', is_success=True)

In [59]:
# | hide
import nbdev

nbdev.nbdev_export()

Bad pipe message: %s [b"\x1e\xbc\xfc\xe6H\xba\x9a\xa5\x0f8\xbfg\xcc\xba\x1d\xb0\xb7>\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0#\xc0'\x00g\x00@\xc0\n\xc0\x14\x009\x008\xc0\t\xc0\x13\x003\x00"]
Bad pipe message: %s [b'\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa0\xc0\x9c\xc0P\x00=\x00<\x005\x00/\x00\x9a\x00\x99\xc0\x07\xc0\x11\x00\x96\x00\x05\x00\xff\x01\x00\x00j\x00\x00\x00\x0e\x00\x0c\x00\x00']
Bad pipe message: %s [b'\x84\x90\x0c\xc8\xa4\xf3\x0e\x0c\t\xd4\xcfp\xa8\x8a\x87\xb4\x8c\x13\x00\x00\xa6\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0s\xc0w\x00\xc4\x00']
Bad pipe message: %s [b"#\xc0'\x00g\x00@\xc0r\xc0v\x00\xbe\x00\xbd\xc0\n\xc0\x14\x009\x008\x00\x88\x00\x8